In [2]:
import os
import MDAnalysis as mda

In [3]:
bead_type_hash = {}
bead_type_hash["G"] = 2.25
bead_type_hash["A"] = 2.52
bead_type_hash["R"] = 3.28
bead_type_hash["K"] = 3.18
bead_type_hash["H"] = 3.04
bead_type_hash["D"] = 2.79
bead_type_hash["E"] = 2.96
bead_type_hash["S"] = 2.59
bead_type_hash["T"] = 2.81
bead_type_hash["N"] = 3.01
bead_type_hash["C"] = 2.74
bead_type_hash["P"] = 2.78
bead_type_hash["I"] = 3.09
bead_type_hash["M"] = 3.09
bead_type_hash["L"] = 3.09
bead_type_hash["F"] = 3.18
bead_type_hash["W"] = 3.39
bead_type_hash["Y"] = 3.23
bead_type_hash["V"] = 2.93

In [4]:
chain = "MASGRP"



In [5]:
def chain2bead(bead_type_hash, chain):
    bead_chain_list = [] #[backbone][side_bead]
    backbone_radius = 1.9
    x = 0
    bead_num = 1
    for bead in chain:
        side_bead_radius = bead_type_hash[bead]
        #backbone
        bead_chain_list.append((x,0,0))
        bead_chain_list.append((x,side_bead_radius+backbone_radius,0))
        x += backbone_radius*2
        bead_num += 1
    return bead_chain_list
test = chain2bead(bead_type_hash, chain)
print(test)
    
    


[(0, 0, 0), (0, 4.99, 0), (3.8, 0, 0), (3.8, 4.42, 0), (7.6, 0, 0), (7.6, 4.49, 0), (11.399999999999999, 0, 0), (11.399999999999999, 4.15, 0), (15.2, 0, 0), (15.2, 5.18, 0), (19.0, 0, 0), (19.0, 4.68, 0)]


In [6]:
def beads2pdb(chain,bead_list):
    atom_count = 1
    filename = "system.pdb"
    id = "A"
    with open(filename, 'w') as file_object:
        
        file_object.write("MODEL        1\n")
        chain_name = 0
        for i, (x, y, z) in enumerate(bead_list):
            
            if i % 2 == 0:
                atom_name = "  CA"
                residue_name = chain[chain_name]
            else:
                atom_name = "  SC"
                residue_name =  chain[chain_name]
                chain_name+=1
            
            residue_index = (i // 2) + 1
            
            #pdb_line = (
                #f"ATOM  {atom_count:5d} {atom_name:<4s} {residue_name:<3s}  {residue_index:4d}"
                #f"{x:8.3f}{y:8.3f}{z:8.3f}"
                #f"\n"
            #)
            pdb_line = (
                f"HETATM{atom_count:5d} {atom_name:<4s} {residue_name:3s} {id:1s}"
                f"{residue_index:4d}    "
                f"{x:8.3f}{y:8.3f}{z:8.3f}\n"
            )
            print(pdb_line)
            
            file_object.write(pdb_line)
            atom_count += 1
            
        file_object.write("TER\n")
        file_object.write("END\n")
        
    print(f"Successfully wrote {atom_count - 1} beads to {filename}")
beads2pdb(chain, test)

HETATM    1   CA M   A   1       0.000   0.000   0.000

HETATM    2   SC M   A   1       0.000   4.990   0.000

HETATM    3   CA A   A   2       3.800   0.000   0.000

HETATM    4   SC A   A   2       3.800   4.420   0.000

HETATM    5   CA S   A   3       7.600   0.000   0.000

HETATM    6   SC S   A   3       7.600   4.490   0.000

HETATM    7   CA G   A   4      11.400   0.000   0.000

HETATM    8   SC G   A   4      11.400   4.150   0.000

HETATM    9   CA R   A   5      15.200   0.000   0.000

HETATM   10   SC R   A   5      15.200   5.180   0.000

HETATM   11   CA P   A   6      19.000   0.000   0.000

HETATM   12   SC P   A   6      19.000   4.680   0.000

Successfully wrote 12 beads to system.pdb


In [19]:
def generate_bond_dat(chain, bead_type_hash, filename="bond_dist.dat"):
    """
    Generates a .dat file with bond distances.
    1. Backbone-Backbone (fixed 3.8)
    2. Backbone-Sidechain (1.9 + side_radius)
    """
    backbone_radius = 1.9
    lines = []
    
    num_residues = len(chain)
    print(len(chain))
    
    for i in range(num_residues-1):
        idx1 = 2 * i
        idx2 = 2 * (i + 1)
        dist = backbone_radius + backbone_radius # Always 3.8
        
        lines.append(f"{idx1}\t {idx2}\t  {dist:.10f}\n")
        
    for i in range(num_residues):
        idx1 = 2 * i
        idx2 = 2 * i + 1
        
        residue_char = chain[i]
        side_radius = bead_type_hash[residue_char]
        dist = backbone_radius + side_radius
        
        lines.append(f"{idx1}\t {idx2}\t  {dist:.10f}\n")

    with open(filename, 'w') as f:
        f.writelines(lines)
    
    print(f"Successfully wrote {len(lines)} bonds to {filename}")

In [20]:
generate_bond_dat(chain, bead_type_hash, "bond_dist.dat")

6
Successfully wrote 11 bonds to bond_dist.dat
